In [140]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: opencv-python in c:\users\sarialop\anaconda3\lib\site-packages (4.7.0.72)



In [141]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf  
from tensorflow import convert_to_tensor              
from tqdm import tqdm
from PIL import Image
import urllib.request

In [142]:
# Cargamos los datos de los ficheros y los mostramos

import json


with open('pantalones.json', 'r') as p:
        pantalones = json.load(p)
print(type(pantalones))
print(len(pantalones))


with open('bicicletas.json', 'r') as b:
        bicicletas = json.load(b)
print(type(bicicletas))
print(len(bicicletas))
with open('televisiones.json', 'r') as t:
        televisiones = json.load(t)
print(type(televisiones))
print(len(televisiones))

with open('camisetas.json', 'r') as c:
        camisetas = json.load(c)
print(type(camisetas))
print(len(camisetas))

<class 'list'>
40
<class 'list'>
40
<class 'list'>
40
<class 'list'>
40


In [143]:
# Hacemos un dataframe con los datos de los dos ficheros

import pandas as pd

tododatos = pantalones + bicicletas + televisiones + camisetas

df = pd.DataFrame(tododatos)
df


,id,title,description,distance,images,user,flags,visibility_flags,price,currency,...,supports_shipping,shipping_allowed,seller_id,favorited,creation_date,modification_date,location,type_attributes,taxonomy,discount
0,wzy50mrwq5j5,Pantalón para de campana cuero marrón,pantalón de cuero de pata de campana de color ...,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'p8j359kvgyz9', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",18.00,EUR,...,True,True,p8j359kvgyz9,False,1678873428724,1678873439187,"{'city': 'Albacete', 'postal_code': '02003', '...","{'brand': {'value': 'Bershka', 'title': 'Marca...",None,None
1,qzmywrkno1zv,pantalones,pantalones vaqueros de campana. Estan en muy b...,2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '8z81807o1r63', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",10.00,EUR,...,True,True,8z81807o1r63,False,1678809627731,1678809637929,"{'city': 'Albacete', 'postal_code': '02002', '...","{'size': {'value': '2', 'title': 'Talla', 'tex...",None,None
2,mzn8xyk91yzn,pantalón,talla 38 están nuevos me equivoque de talla,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'npj9k4v9kze7', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",5.00,EUR,...,True,True,npj9k4v9kze7,False,1678808142425,1678808152827,"{'city': 'Albacete', 'postal_code': '02005', '...","{'gender': {'value': 'female', 'title': 'Géner...",None,None
3,e65glw2oypjo,pantalón,"nuevo en perfecto estado, talla M",56.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'p8j3e907m7z9', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",3.00,EUR,...,True,True,p8j3e907m7z9,False,1678791766067,1678791776205,"{'city': 'Hellín', 'postal_code': '02400', 'co...","{'brand': {'value': 'Primark', 'title': 'Marca...",None,None
4,vjrr300ny4jk,Pantalon Levi's 511,"Pantalon Levi's original en perfecto estado, g...",1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '9nz0r8dre7zo', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",30.00,EUR,...,True,True,9nz0r8dre7zo,False,1678818895922,1678819425058,"{'city': 'Albacete', 'postal_code': '02005', '...","{'brand': {'value': 'Levi'S', 'title': 'Marca'...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0j23l0wmee6y,camiseta niña,Lote por 6. Talla 9 pero da una 7.\n,2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'wzvyl1pyldzl', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",6.00,EUR,...,True,True,wzvyl1pyldzl,False,1678832873903,1678832884052,"{'city': 'Albacete', 'postal_code': '02002', '...",None,None,None
156,wzy503ryxpj5,2 camisetas verano,"Son bastante cómodas, se amolda a el cuerpo y ...",2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '8z818lpemr63', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",17.00,EUR,...,True,True,8z818lpemr63,False,1678742790336,1678742800509,"{'city': 'Albacete', 'postal_code': '02004', '...","{'brand': {'value': 'Bershka', 'title': 'Marca...",None,None
157,pj95l11lgvze,camiseta del Valencia,dañado por la zona del cuello,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '0j2y4717rdzy', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",8.00,EUR,...,True,True,0j2y4717rdzy,False,1678715578118,1678715588264,"{'city': 'Albacete', 'postal_code': '02005', '...","{'brand': {'value': 'Adidas',

In [144]:
# Eliminamos las columnas que no nos interesan
df_todosdatos = df.drop(['price', 'description'], axis=1)

df_todosdatos
# Aplicamos la función que hemos creado en el paso anterior a la columna title del dataframe
df_todosdatos['title'] = df_todosdatos.apply(lambda x: x['title'].lower(), axis=1)

df_todosdatos

,id,title,distance,images,user,flags,visibility_flags,currency,free_shipping,web_slug,...,supports_shipping,shipping_allowed,seller_id,favorited,creation_date,modification_date,location,type_attributes,taxonomy,discount
0,wzy50mrwq5j5,pantalón para de campana cuero marrón,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'p8j359kvgyz9', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,pantalon-para-de-campana-cuero-marron-883281244,...,True,True,p8j359kvgyz9,False,1678873428724,1678873439187,"{'city': 'Albacete', 'postal_code': '02003', '...","{'brand': {'value': 'Bershka', 'title': 'Marca...",None,None
1,qzmywrkno1zv,pantalones,2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '8z81807o1r63', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,pantalones-883111308,...,True,True,8z81807o1r63,False,1678809627731,1678809637929,"{'city': 'Albacete', 'postal_code': '02002', '...","{'size': {'value': '2', 'title': 'Talla', 'tex...",None,None
2,mzn8xyk91yzn,pantalón,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'npj9k4v9kze7', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,pantalon-883104597,...,True,True,npj9k4v9kze7,False,1678808142425,1678808152827,"{'city': 'Albacete', 'postal_code': '02005', '...","{'gender': {'value': 'female', 'title': 'Géner...",None,None
3,e65glw2oypjo,pantalón,56.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'p8j3e907m7z9', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,pantalon-883032248,...,True,True,p8j3e907m7z9,False,1678791766067,1678791776205,"{'city': 'Hellín', 'postal_code': '02400', 'co...","{'brand': {'value': 'Primark', 'title': 'Marca...",None,None
4,vjrr300ny4jk,pantalon levi's 511,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '9nz0r8dre7zo', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,pantalon-levi-s-511-883158919,...,True,True,9nz0r8dre7zo,False,1678818895922,1678819425058,"{'city': 'Albacete', 'postal_code': '02005', '...","{'brand': {'value': 'Levi'S', 'title': 'Marca'...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0j23l0wmee6y,camiseta niña,2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'wzvyl1pyldzl', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,camiseta-nina-883215059,...,True,True,wzvyl1pyldzl,False,1678832873903,1678832884052,"{'city': 'Albacete', 'postal_code': '02002', '...",None,None,None
156,wzy503ryxpj5,2 camisetas verano,2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '8z818lpemr63', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,2-camisetas-verano-882943120,...,True,True,8z818lpemr63,False,1678742790336,1678742800509,"{'city': 'Albacete', 'postal_code': '02004', '...","{'brand': {'value': 'Bershka', 'title': 'Marca...",None,None
157,pj95l11lgvze,camiseta del valencia,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '0j2y4717rdzy', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,camiseta-del-valencia-882808785,...,True,True,0j2y4717rdzy,False,1678715578118,1678715588264,"{'city': 'Albacete', 'postal_code': '02005', '...","{'brand': {'value': 'Adidas', 'title': 'Marca'...",None,None
158,8j3el0end7z9,camiseta canibal corpse,2.0,[{'original': 'http://cdn.wallapop.com/imag

In [145]:
url = df_todosdatos['images'].iloc[0]
print(url)

[{'original': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W800', 'xsmall': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W320', 'small': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W320', 'large': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W800', 'medium': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W640', 'xlarge': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W800', 'original_width': 0, 'original_height': 0}, {'original': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180295.jpg?pictureSize=W800', 'xsmall': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180295.jpg?pictureSize=W320', 'small': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i31911

In [146]:
# Creamos una función que nos permita cargar las imágenes
def load_image(row):
  try:
    imagen = urllib.request.urlopen(row['images'])
    img_pil = Image.open(imagen)
    img_np = np.array(img_pil)
    img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    
    return img_np
  except: return None

In [147]:
imagen = urllib.request.urlopen("http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W800http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W800")
img_pil = Image.open(imagen)
img_np = np.array(img_pil)

print(img_np)

[[[179 183 184]
  [179 183 184]
  [180 184 185]
  ...
  [179 179 179]
  [179 179 179]
  [179 179 179]]

 [[180 184 185]
  [180 184 185]
  [180 184 185]
  ...
  [179 179 179]
  [179 179 179]
  [179 179 179]]

 [[180 184 185]
  [180 184 185]
  [180 184 185]
  ...
  [180 180 180]
  [179 179 179]
  [179 179 179]]

 ...

 [[145 135 126]
  [145 135 126]
  [145 135 126]
  ...
  [146 141 135]
  [146 141 135]
  [146 141 135]]

 [[145 135 126]
  [145 135 126]
  [145 135 126]
  ...
  [146 141 135]
  [146 141 135]
  [146 141 135]]

 [[144 134 125]
  [145 135 126]
  [145 135 126]
  ...
  [147 142 136]
  [147 142 136]
  [146 141 135]]]


In [148]:
# Aplicamos la función que hemos creado en el paso anterior a la columna images del dataframe para descargar todas las imágenes
df_todosdatos['images'] = df_todosdatos.apply(load_image, axis=1)
